In [ ]:
import numpy as np

class RandomProcess:
    def __init__(self):
        pass

class Particle:
    def __init__(self,):
        self.present_position: np.ndarray # Dimension of the space.
        self.present_velocity: np.ndarray 
        self.present_time: int

    def Init_Random_Speed(self):
        self.present_velocity = np.random.randn(self.present_position.shape[0])
        pass
    
    def Init_Required_Speed(self,speed):
        self.present_velocity = speed
        pass

    def move(self,acceleration):
        